# Trade Network Analysis
`08_imports_decomp.ipynb`

Requires:

1. `fat_tailed_prod.csv` created by `03_deg_dist.ipynb`
2. `emergency_countries` created by `05_em-dat.ipynb`
3. Cleaned trade data for each year created by `02_clean.ipynb`

Produces a csv file with the USD value (in thousands) of each countries imports for each year decomposed into four categories.

* cs: value of imports of products with fat-tailed distribution of exporter sizes imported from countries with a natural disaster in the next year. The c stands for 'central players' and the s stands for 'supply shock'.
* ns: value of imports of products with more normal distribution of exporter sizes imported from countries with a natural disaster in the next year.
* cn: value of imports of products with fat-tailed distribution of exporter sizes imported from countries with no major natural disaster in the next year.
* nn: value of imports of products with more normal distribution of exporter sizes imported from countries with no major natural disaster in the next year.


---

METODO:
1. Annotate 
2. Add text above with formulas
3. Calculate percentage of total

#### Libraries

In [1]:
import pandas as pd                                         # pandas dataframes used for convenience
import os                                                   # change current directory in next line
os.chdir('C:/Working/trade_network/data/')

#### Load distribution and natural disaster data

Load distribution and natural disaster dummy variable datasets from csv files.

In [2]:
fat_tailed_prod = pd.read_csv('fat_tailed_prod.csv', index_col='year')
emer_c = pd.read_csv('emergency_countries.csv', index_col='y')
emer_c = emer_c.rename(columns={'iso': 'i'})

#### Decompose imports for each country and year

The dummy variable sets from above are joined to the cleaned trade data. The value of exports is then multiplied by the dummy data so bilateral trade values are each put into one the four buckets described in the header. The results are summed by country and saved as an annual csv file.

In [3]:
for y in range(2008,2015):         # start year & end year + 1 
    ftp = fat_tailed_prod.loc[y].reset_index()[['hs6','c']].set_index('hs6')
    ec = emer_c.loc[y].reset_index()[['i','s']].set_index('i')
    base_file = pd.read_csv('clean/baci07_'+str(y)+'_clean.csv')[['hs6','i','j','v']]
    df = base_file.join(ec, on='i').join(ftp, on='hs6').fillna(value=0)
    df['cs'] = (df['v'] * df['c'] * df['s'])                    
    df['ns'] = (df['v'] * (1 - df['c']) * df['s'])
    df['cn'] = (df['v'] * (1 - df['s']) * df['c'])
    df['nn'] = (df['v'] * (1 - df['s']) * (1 - df['c']))
    df.groupby(['j']).sum()[['cs','ns','cn','nn']].to_csv('summary/imports_decomp_'+str(y)+'.csv')

#### Combine to one file

This just puts together the annual csv files.

In [4]:
df = {}
for y in range(2008,2015):         # start year & end year + 1 '
    df[y] = pd.read_csv('summary/imports_decomp_'+str(y)+'.csv', index_col='j')
pd.concat(df, axis=0).to_csv('imports_decomp.csv')